In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('/Users/carterhogan/CaseStudies/world_value_survey/analysis/data/wvs/WVS_Time_Series.csv')

In [2]:
# We want to now change political preference to a factored variable
pol_pref = ["Q240"]

# we need to remove all pol pref values less than 0





0

In [4]:
# baseline happiness average
avg_hap = df['Q46'].mean()
avg_hap

unique_hap = df['Q46'].unique()
unique_hap

# 

array([0.66666667, 0.33333333, 1.        , 0.        ])